In [1]:
import torch
import clip
import os

import numpy as np

from src.imagenet_labels import lab_dict
from tqdm.notebook import tqdm
from src.dataloaders import imagenet_c_dataloader, imagenet_dataloader

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model, transform = clip.load("ViT-B/32", device="cpu")

In [4]:
cls_names = [lab_dict[i].replace('_', ' ') for i in os.listdir('../data/imagenet')]
cls_names = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cls_names]).to("cpu")

In [5]:
text_features = model.encode_text(cls_names)
text_features /= text_features.norm(dim=-1, keepdim=True)

In [6]:
model = model.to(device)

In [11]:
def get_acc(gt, preds = None):
    if preds is not None: 
        return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
    return ((preds.argmax(1)==gt).sum()/len(preds)).cpu().numpy()
    

def get_test_acc(model, loader, device='cuda'):
    eval_acc = []
    for batch in tqdm(loader, leave=False):
        ims, labels = batch
        ims, labels = ims.to(device), labels.to("cpu")
        with torch.no_grad():
            image_features = model.encode_image(ims)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            image_features = image_features.to("cpu")
            probs = (image_features @ text_features.T).softmax(dim=-1)
            
        val_acc = get_acc(labels.view(-1,), probs)
        eval_acc.append(val_acc)
    
    return np.mean(eval_acc)

In [10]:
gaussian_noise_acc = []
for sev in tqdm([1, 2, 3, 4, 5]):
    loader = imagenet_c_dataloader(corruption_name='gaussian_noise', severity=sev, batch_size=256, transform=transform)
    gaussian_noise_acc.append(get_test_acc(model, loader, device))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

In [12]:
gaussian_noise_acc

[0.5286392, 0.47575733, 0.38308355, 0.26073024, 0.1320751]

In [13]:
impulse_noise_acc = []
for sev in tqdm([1, 2, 3, 4, 5]):
    loader = imagenet_c_dataloader(corruption_name='impulse_noise', severity=sev, batch_size=256, transform=transform)
    impulse_noise_acc.append(get_test_acc(model, loader, device))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

In [14]:
impulse_noise_acc

[0.4705078, 0.41455278, 0.36472017, 0.25004384, 0.1365633]

In [15]:
shot_noise_acc = []
for sev in tqdm([1, 2, 3, 4, 5]):
    loader = imagenet_c_dataloader(corruption_name='shot_noise', severity=sev, batch_size=256, transform=transform)
    shot_noise_acc.append(get_test_acc(model, loader, device))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

In [16]:
shot_noise_acc

[0.5197026, 0.4564533, 0.3620695, 0.2157685, 0.1340322]

In [17]:
speckle_noise_acc = []
for sev in tqdm([1, 2, 3, 4, 5]):
    loader = imagenet_c_dataloader(corruption_name='speckle_noise', severity=sev, batch_size=256, transform=transform)
    speckle_noise_acc.append(get_test_acc(model, loader, device))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

In [18]:
speckle_noise_acc

[0.52785397, 0.4948302, 0.37495616, 0.29706234, 0.21020408]

In [19]:
loader = imagenet_dataloader(batch_size=256, transform=transform)
clean_acc = get_test_acc(model, loader, device)

  0%|          | 0/196 [00:00<?, ?it/s]

In [20]:
clean_acc

0.59108335